# Лабораторная работа  "Подсчет кэш-промахов"

### 1. Определим параметры кэшей CPU c помощью утилиты cpuid:

```
akasiyanik@akasiyanik-pc:~/Dev/fpmi/cache-miss-lab$ cpuid
...
 cache and TLB information (2):
      0xb1: instruction TLB: 2M/4M, 4-way, 4/8 entries
      0xb0: instruction TLB: 4K, 4-way, 128 entries
      0x05: data TLB: 4M pages, 4-way, 32 entries
      0xf0: 64 byte prefetching
      0x57: L1 data TLB: 4K pages, 4-way, 16 entries
      0x56: L1 data TLB: 4M pages, 4-way, 16 entries
      0x7d: L2 cache: 2M, 8-way, sectored, 64 byte lines
      0x30: L1 cache: 32K, 8-way, 64 byte lines
      0xb4: data TLB: 4K pages, 4-way, 256 entries
      0x2c: L1 data cache: 32K, 8-way, 64 byte lines
...
```

### 2. Определим параметры десктопа.
Параметры CPU:

```akasiyanik@akasiyanik-pc:~$ lscpu
Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                4
On-line CPU(s) list:   0-3
Thread(s) per core:    1
Core(s) per socket:    4
Socket(s):             1
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 23
Model name:            Intel(R) Core(TM)2 Quad CPU    Q8400  @ 2.66GHz
Stepping:              10
CPU MHz:               2000.000
CPU max MHz:           2667.0000
CPU min MHz:           2000.0000
BogoMIPS:              5332.93
Virtualization:        VT-x
```


Общие параметры:
![title](./system_info.png)

### 3. Код анализа количества кэш промахов

[main.cpp](file/main.cpp)

```cpp
class Cache {
    public:
        long long hitCounter = 0;
        long long missCounter = 0;

        Cache(int cacheLineSize, int channels, int size) : cacheLineSize(cacheLineSize), channels(channels),
                                                           size(size) {
            totalLinesInCache = size / cacheLineSize;
            linesInChannel = totalLinesInCache / channels;

            cout << "cache size " << size << " B" << endl;
            cout << "channels " << channels << endl;
            cout << "cacheLineSize " << cacheLineSize << " B" << endl;
            cout << "totalLinesInCache " << totalLinesInCache << endl;
            cout << "linesInChannel " << linesInChannel << endl;

            cache = new long long[totalLinesInCache];
            lastAccessTime = new long long[totalLinesInCache];
            for (int i = 0; i < totalLinesInCache; i++) {
                cache[i] = -1;
                lastAccessTime[i] = -1;
            }
        }

        virtual ~Cache() {
            delete[] cache;
            delete[] lastAccessTime;

        }

        void access(long long addrInMem) {
            long long lineNumberInMem = getCacheLineNumber(addrInMem);
            int lineTag = (int) (lineNumberInMem % linesInChannel);

            if (isCacheContain(lineTag, lineNumberInMem)) {
                hitCounter++;
            } else {
                missCounter++;
                addLineToCache(lineTag, lineNumberInMem);
            }
            return;
        }

    private:
        int cacheLineSize;
        int channels;
        int size;

        int linesInChannel;
        int totalLinesInCache;

        long long *cache;
        long long *lastAccessTime;

        long long getCurrentTimestamp() {
            return chrono::system_clock::now().time_since_epoch().count();
        }

        bool isCacheContain(int cacheLineTag, long long cacheLineNumberInMem) {
            for (int i = 0; i < channels; i++) {
                long n = cacheLineTag * channels + i;
                if (cache[n] == cacheLineNumberInMem) {
                    lastAccessTime[n] = getCurrentTimestamp();
                    return true;
                }
            }
            return false;
        }

        void addLineToCache(int cacheLineTag, long long cacheLineNumberInMem) {
            long lruLineNum = cacheLineTag * channels;
            long minAccessTime = lastAccessTime[lruLineNum];
            for (int i = 1; i < channels; i++) {
                long n = cacheLineTag * channels + i;
                if (lastAccessTime[n] < minAccessTime) {
                    minAccessTime = lastAccessTime[n];
                    lruLineNum = n;
                }
            }

            cache[lruLineNum] = cacheLineNumberInMem;
            lastAccessTime[lruLineNum] = getCurrentTimestamp();
        }

        long long getCacheLineNumber(long long addrInMem) {
            return addrInMem / cacheLineSize;
        }
    };

```

### 4. Графики сравнения количества кэш-промахов, предсказанных valgrind и собственным анализатором